In [1]:

import os
import torch
from ultralytics import YOLO
from roboflow import Roboflow

# Check available GPUs
num_gpus = torch.cuda.device_count()
device_ids = list(range(num_gpus))  # Example: [0,1,2,3] if 4 GPUs are available

# Define dataset path (update if needed)
dataset_path = "/home/mansi/Sumit Kumar, Satyendra/Eyes Final fgs/eyepac 5"
data_yaml = os.path.join(dataset_path, "data.yaml")

# Check if dataset exists
if not os.path.exists(data_yaml):
    print("Dataset not found! Downloading...")
    rf = Roboflow(api_key="esmnGAg3ff1gA7CI0fMF")
    project = rf.workspace("eye-pac-3").project("eye-pac-new-2")
    version = project.version(4)
    dataset = version.download("yolov8")
    os.rename(dataset.location, dataset_path)
else:
    print("Dataset already exists. Using the existing dataset.")

# Load YOLO model
model = YOLO("yolov8n")  # Adjust model if needed




Dataset not found! Downloading...
loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Eye-pac-new-2-4 in yolov8:: 100%|████████████████████████████████████████████████| 321112/321112 [05:23<00:00, 992.21it/s]


In [2]:
import os
from collections import defaultdict

# Base dataset path
dataset_path = "/home/mansi/Sumit Kumar, Satyendra/Eyes Final fgs/eyepac 5"

# Corrected label paths
labels_dirs = [
    os.path.join(dataset_path, "valid", "labels"),
    os.path.join(dataset_path, "test", "labels"),
]

# Dictionary to track images per class
class_to_images = defaultdict(set)

# Iterate and count
for labels_dir in labels_dirs:
    if not os.path.exists(labels_dir):
        print(f"Missing directory: {labels_dir}")
        continue

    print(f"Scanning: {labels_dir}")
    for label_file in os.listdir(labels_dir):
        if not label_file.endswith(".txt"):
            continue

        label_path = os.path.join(labels_dir, label_file)

        with open(label_path, "r") as f:
            lines = f.readlines()
            for line in lines:
                if line.strip() == "":
                    continue
                class_id = int(line.split()[0])
                class_to_images[class_id].add(label_file)

# Output counts
print("\nNumber of images per class:")
for class_id in range(5):  # Classes 0 to 4
    count = len(class_to_images[class_id])
    print(f"Class {class_id}: {count} images")


Scanning: /home/mansi/Sumit Kumar, Satyendra/Eyes Final fgs/eyepac 5/valid/labels
Scanning: /home/mansi/Sumit Kumar, Satyendra/Eyes Final fgs/eyepac 5/test/labels

Number of images per class:
Class 0: 5162 images
Class 1: 4397 images
Class 2: 6350 images
Class 3: 1224 images
Class 4: 713 images


In [3]:
# Train with optimized settings
model.train(
    data=data_yaml,
    name="eyepac balanced aug 5",
    device=device_ids,  # Use all available GPUs
    epochs=100, 
    batch=64,  # Adjust batch size for better GPU utilization
    workers=16,  # Use multiple data loading workers
#     optimizer="SGD"  # Use SGD optimizer
)

Ultralytics 8.3.99 🚀 Python-3.8.13 torch-2.4.1+cu121 CUDA:0 (NVIDIA GeForce RTX 2080 Ti, 11012MiB)
                                                      CUDA:1 (NVIDIA GeForce RTX 2080 Ti, 11012MiB)
                                                      CUDA:2 (NVIDIA GeForce RTX 2080 Ti, 11012MiB)
                                                      CUDA:3 (NVIDIA GeForce RTX 2080 Ti, 11012MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/home/mansi/Sumit Kumar, Satyendra/Eyes Final fgs/eyepac 5/data.yaml, epochs=100, time=None, patience=100, batch=64, imgsz=640, save=True, save_period=-1, cache=False, device=[0, 1, 2, 3], workers=16, project=None, name=eyepac balanced aug 5, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, s

train: Scanning /home/mansi/Sumit Kumar, Satyendra/Eyes Final fgs/eyepac 5/train/labels... 142704 images, 0 backgrounds, 0 corrupt: 100%|██████████| 142704/142704 [04:50<00:00, 492.08it/s]


train: New cache created: /home/mansi/Sumit Kumar, Satyendra/Eyes Final fgs/eyepac 5/train/labels.cache


val: Scanning /home/mansi/Sumit Kumar, Satyendra/Eyes Final fgs/eyepac 5/valid/labels... 13373 images, 0 backgrounds, 0 corrupt: 100%|██████████| 13373/13373 [00:31<00:00, 420.66it/s]


val: New cache created: /home/mansi/Sumit Kumar, Satyendra/Eyes Final fgs/eyepac 5/valid/labels.cache
Plotting labels to runs/detect/eyepac balanced aug 5/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: SGD(lr=0.01, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 64 dataloader workers
Logging results to runs/detect/eyepac balanced aug 5
Starting training for 100 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

     94/100      2.47G    0.07723      1.147     0.8605         12        640: 100%|██████████| 2230/2230 [05:50<00:00,  6.36it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 418/418 [00:49<00:00,  8.46it/s]


                   all      13373      13373      0.603      0.398      0.134     0.0137

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


     95/100      2.47G     0.0591      1.124     0.8485         12        640: 100%|██████████| 2230/2230 [05:50<00:00,  6.37it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 418/418 [00:48<00:00,  8.57it/s]


                   all      13373      13373      0.603      0.398      0.156     0.0158

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


     96/100      2.47G        inf      1.113     0.8458         16        640:  94%|█████████▎| 2089/2230 [05:28<00:22,  6.40it/s]IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [4]:
# Validation
print("Running validation...")
metrics = model.val()
print("Validation Results:", metrics)

Running validation...
Ultralytics 8.3.99 🚀 Python-3.8.13 torch-2.4.1+cu121 CUDA:0 (NVIDIA GeForce RTX 2080 Ti, 11012MiB)
                                                      CUDA:1 (NVIDIA GeForce RTX 2080 Ti, 11012MiB)
                                                      CUDA:2 (NVIDIA GeForce RTX 2080 Ti, 11012MiB)
                                                      CUDA:3 (NVIDIA GeForce RTX 2080 Ti, 11012MiB)
Model summary (fused): 72 layers, 3,006,623 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning /home/mansi/Sumit Kumar, Satyendra/Eyes Final fgs/eyepac 5/valid/labels.cache... 13373 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1337
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 209/209 [01:09<00:00,  3.00it/s]


                   all      13373      13373      0.806      0.809      0.879      0.878
                     0       3872       3872      0.757      0.946      0.917      0.916
                     1       3298       3298      0.834      0.737      0.866      0.866
                     2       4762       4762        0.8      0.854       0.91       0.91
                     3        916        916      0.732      0.774      0.827      0.827
                     4        525        525      0.908      0.733      0.873      0.873
Speed: 0.1ms preprocess, 1.1ms inference, 0.0ms loss, 0.8ms postprocess per image
Results saved to runs/detect/eyepac balanced aug 52
Validation Results: ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3, 4])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7fd0e76eb250>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 

In [ ]:
# Testing
print("Running testing...")
test_results = model.val(split="test")
print("Test Results:", test_results)